# Datenanalyse der Brettspielwelt-DB

## 1. Datenbestand

Es stehen ca. 7 GB Logdateien vom Spiele-Portal "Brettspielwelt" zur Verfügung (seit 2007-01, alle Logdateien bis einschließlich 2025-07).
Jede Logdatei hat die Daten einer Partie.

Nach dem Download wurde jede Logdatei geparst (in ein Objekt eingelesen), validiert und in eine SQLite-DB gepumpt.
Dieser Vorgang dauerte ca. 20 Stunden (wobei die Indezies der DB aus Versehen bereits vor dem Import angelegt wurden - dadurch hat es schätzungsweise 4 Stunden länger gedauert):

```
C:\Users\frank\Source\PyCharm\tichu\.venv\Scripts\python.exe C:\Users\frank\Source\PyCharm\tichu\bin\bsw_import.py --ym1 2007-01 --ym2 2025-07
BSW Importer
Ab Datum: 2007-01
Bis Datum: 2025-07
Zip-Archiven: C:\Users\frank\Source\PyCharm\tichu\data\bsw\tichulog
SQLite-Datenbank: C:\Users\frank\Source\PyCharm\tichu\data\bsw\bsw.sqlite
Importiere Log-Dateien: 100%|██████████| 2411514/2411514 [19:47:30<00:00, 33.85 Datei/s, Logdateien=2411514, Leere=13964, Fehler=79663, Datei=202507/2411561.tch]
Aggregiere Daten für Spieler: 100%|██████████| 128462/128462 [42:27<00:00, 50.43 Spieler/s]
fertig

Process finished with exit code 0
```

Dabei wurde gezählt:
*   2.411.514 Logdateien insgesamt
*   13.964 leere Logdateien (ohne verwertbare Daten)

Die SQLite-Datenbankdatei ist 17.085.088 KB = 16,294 GB groß.

### 1.1 Anzahl Partien

*   2.397.550 Partien insgesamt
*   2.317.858 Partien fehlerfrei

In [1]:
%%sql
-- Anzahl Partien gesamt
select count(*) as Partien from games;

,Partien
0,2397550


In [1]:
%%sql
-- Anzahl Partien groupiert nach Fehlern
select g.error_code as Code, e.name as Fehler, count(*) as Anzahl, e.description as Beschreibung
from games as g
left join errors as e on g.error_code = e.code
group by g.error_code
order by g.error_code;

,Code,Fehler,Anzahl,Beschreibung
0,0,NO_ERROR,2317858,Kein Fehler
1,70,GAME_NOT_FINISHED,66718,Partie nicht zu Ende gespielt.
2,71,GAME_OVERPLAYED,1959,"Ein oder mehrere Runden gespielt, obwohl die P..."
3,80,ROUND_FAILED,11015,Mindestens eine Runde ist fehlerhaft.


In [3]:
%%sql
-- Anzahl Partien mit Spielerwechsel
select count(*) as Partien from games where player_changed = 1

,Partien
0,160943


Wurde weitergespielt, wegen Unentschieden?

Nein, nicht deswegen. In zwei Spielen wurde nach Unentschieden über 1000 Punkte zwar weitergespielt, aber in beiden wurde auch vorher schon über das Ziel hinaus weitergespielt, und da stand eindeutig des Siegerteam fest.

ALso, Gleichstand war nicht der Grund.

In [2]:
%%sql
-- Wegen Unentschieden weitergespielt
select * from rounds where score_cum_20 == score_cum_31 and score_cum_20 >= 1000

,id,game_id,round_index,score_cum_20,score_cum_31,wish_value,dragon_giver,dragon_recipient,gift_relative_index,is_phoenix_low,winner_index,winner_position,loser_index,is_double_victory,score_20,score_31,score_diff,history,error_code,error_context
0,2594118,264153,16,1300,1300,2,3,0,1,0,2,16,0,0,240,60,180,2:SB|3:BK|0|1|2:BA|3|0|1;2|2:G9R8B7G6R5G4G3PhM...,0,NaN
1,2623727,267189,14,1050,1050,2,3,0,1,0,2,27,0,0,235,65,170,2:Ma|3:S2|0:B3|1:R5|2:RA|3|0|1;2|2:Hu|0:RZ|1|2...,0,NaN


### 1.2 Anzahl Runden

*   22.458.253 Runden insgesamt.
*   22.437.758 Runden fehlerfrei
*   22.000.517 Runden unter den fehlerfreien Partien (durchschnittlich 9,5 Runden / Partie)
*   88.002.068 "Perspektiv-Runden"

In [5]:
%%sql
-- Anzahl Runden gesamt
select count(*) as Runden from rounds;

,Runden
0,22458253


In [3]:
%%sql
-- Anzahl Runden groupiert nach Fehlern
select r.error_code as Code, e.name as Fehler, count(*) as Anzahl, e.description as Beschreibung
from rounds as r
left join errors as e on r.error_code = e.code
group by r.error_code
order by r.error_code;

,Code,Fehler,Anzahl,Beschreibung
0,0,NO_ERROR,22437758,Kein Fehler
1,12,DUPLICATE_CARD,10,Karten mehrmals vorhanden.
2,21,WISH_NOT_FOLLOWED,209,Wunsch nicht beachtet.
3,23,PLAYER_NOT_ON_TURN,2,Der Spieler ist nicht am Zug.
4,25,HISTORY_TOO_LONG,60,"Karten ausgespielt, obwohl die Runde vorbei is..."
5,31,DRAGON_GIVEN_TO_OWN_TEAM,306,Drache an eigenes Team verschenkt.
6,50,ANNOUNCEMENT_NOT_POSSIBLE,286,Tichu-Ansage an der geloggten Position nicht m...
7,60,SCORE_NOT_POSSIBLE,17258,Rechenfehler! Geloggtes Rundenergebnis ist nic...
8,61,SCORE_MISMATCH,2364,Geloggtes Rundenergebnis stimmt nicht mit dem ...


In [7]:
%%sql
-- Anzahl Fälle, in denen der Phönix den niedrigeren Rang mimt
select count(*) as Runden from rounds where is_phoenix_low = 1

,Runden
0,21267


In [8]:
%%sql
-- Tichu-Ansage, obwohl ein Mitspieler bereits fertig ist
select count(*) as Ansagen from players_rounds where is_tichu_suicidal = 1

,Ansagen
0,2034


In [4]:
%%sql
-- Anzahl Runden unter den fehlerfreien Partien
select count(*) as Runden
from rounds as r
inner join games as g on g.id = r.game_id
where g.error_code = 0;

,Runden
0,22000148


In [5]:
# Runden unter fehlerfreien Partien / fehlerfreie Partien
22000148 / 2317858

9.491585765823446

In [6]:
# "Perspektiv-Runden" = Runden unter fehlerfreien Partien * 4
22000148 * 4

88000592

### 1.3 Anzahl Spieler

*   128.462 Spieler insgesamt
*   125.646 Spieler, die eine fehlerfreie Partie durchgespielt haben

In [12]:
%%sql
-- Anzahl Spieler gesamt
select count(*) as Spieler from players;

,Spieler
0,128462


In [7]:
%%sql
-- Anzahl Spieler, die eine fehlerfreie Partie durchgespielt haben
select count(distinct pr.player_id) as Spieler
from games as g
inner join rounds as r on g.id = r.game_id
inner join main.players_rounds as pr on r.id = pr.round_id
where g.error_code = 0


,Spieler
0,125646


### 1.4 Anzahl Spielzüge und Stiche

Über alle fehlerfreien Partien:
*   11,0 Stiche pro Runde
*   56,445 Spielzüge pro Runde
*   1.241.791.952 Spielzüge


Im Spiel mit 4 HeuristicAgents fallen ca. 10.8 Stiche/Runde (so häufig wurden Karten kassiert).


In [8]:
%%sql
select
   sum(avg_tricks_per_round * num_rounds) / sum(num_rounds) as Stiche_pro_Runde,
   sum(avg_turns_per_round * num_rounds) / sum(num_rounds) as Spielzuege_pro_Runde,
   sum(avg_turns_per_round * num_rounds) as Spielzuege
from games
where error_code = 0

,Stiche_pro_Runde,Spielzuege_pro_Runde,Spielzuege
0,11.014755,56.444709,1241791952
